<a href="https://colab.research.google.com/github/parkrye/Python/blob/main/202210_Bigdata/SparkSQL_%EC%8B%A4%EC%8A%B5_07_Spark_SQL%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_trip_count_by_zone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
directory="C:\\Users\\mhso_lec\\study_notebook\\data"
trip_file = "fhvhv_tripdata_2020-03.csv"
zone_file = "taxi+_zone_lookup.csv"

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("trip_count_sql").getOrCreate()

In [ ]:
trip_data = spark.read.csv(f"file:///{directory}\\{trip_file}", inferSchema=True, header=True)
zone_data = spark.read.csv(f"file:///{directory}\\{zone_file}", inferSchema=True, header=True)

In [ ]:
trip_data.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   null|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   null|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   null|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   null|
+-----------------+--------------------+-------------------+-------------------+

In [ ]:
zone_data.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [ ]:
trip_data.createOrReplaceTempView("trip_data")
zone_data.createOrReplaceTempView("zone_data")

- PULocationID ( Pick Up Location ID ) : 승차 Location ID
- DOLocationID ( Drop Off Location ID ) : 하차 Location ID

In [ ]:
# 승차 Location(PULocationID)별 개수 세기
query = """

select borough, count(*) as trips
from (
        select zone_data.Borough as borough
        from trip_data
        join zone_data on trip_data.PULocationID = zone_data.LocationID )
        
group by borough

"""
spark.sql(query).show()

+-------------+-------+
|      borough|  trips|
+-------------+-------+
|       Queens|2437383|
|          EWR|    362|
|      Unknown|    845|
|     Brooklyn|3735764|
|Staten Island| 178818|
|    Manhattan|4953140|
|        Bronx|2086592|
+-------------+-------+



In [ ]:
# 하차 Location(DOLocationID)별 개수 세기
query = """

select borough, count(*) as trips
from (
        select zone_data.Borough as borough
        from trip_data
        join zone_data on trip_data.DOLocationID = zone_data.LocationID )
        
group by borough

"""
spark.sql(query).show()

+-------------+-------+
|      borough|  trips|
+-------------+-------+
|       Queens|2468408|
|          EWR|  65066|
|      Unknown| 387759|
|     Brooklyn|3696682|
|Staten Island| 177727|
|    Manhattan|4553776|
|        Bronx|2043486|
+-------------+-------+



In [ ]:
query = """
    
select zone_data.Zone, count(*) as trips

from trip_data
join zone_data on trip_data.PULocationID = zone_data.LocationID

where trip_data.hvfhs_license_num = "HV0003"

group by zone_data.Zone order by trips desc
    
"""

spark.sql(query).show()

+--------------------+------+
|                Zone| trips|
+--------------------+------+
| Crown Heights North|163091|
|       East New York|134198|
|         JFK Airport|114179|
|        East Village|112017|
|      Bushwick South|110150|
|Central Harlem North|108070|
|   LaGuardia Airport|104119|
|Washington Height...| 97324|
|Flatbush/Ditmas Park| 95724|
|            Canarsie| 94484|
|TriBeCa/Civic Center| 94155|
|             Astoria| 92676|
|             Bedford| 90352|
|      Midtown Center| 90261|
|  Stuyvesant Heights| 88749|
|            Union Sq| 88372|
|Times Sq/Theatre ...| 86870|
|Prospect-Lefferts...| 84347|
|         Brownsville| 82764|
|Mott Haven/Port M...| 82396|
+--------------------+------+
only showing top 20 rows



In [ ]:
spark.sql(query).explain(True)

== Parsed Logical Plan ==
'Sort ['trips DESC NULLS LAST], true
+- 'Aggregate ['zone_data.Zone], ['zone_data.Zone, 'count(1) AS trips#244]
   +- 'Filter ('trip_data.hvfhs_license_num = HV0003)
      +- 'Join Inner, ('trip_data.PULocationID = 'zone_data.LocationID)
         :- 'UnresolvedRelation [trip_data], [], false
         +- 'UnresolvedRelation [zone_data], [], false

== Analyzed Logical Plan ==
Zone: string, trips: bigint
Sort [trips#244L DESC NULLS LAST], true
+- Aggregate [Zone#48], [Zone#48, count(1) AS trips#244L]
   +- Filter (hvfhs_license_num#16 = HV0003)
      +- Join Inner, (PULocationID#20 = LocationID#46)
         :- SubqueryAlias trip_data
         :  +- View (`trip_data`, [hvfhs_license_num#16,dispatching_base_num#17,pickup_datetime#18,dropoff_datetime#19,PULocationID#20,DOLocationID#21,SR_Flag#22])
         :     +- Relation [hvfhs_license_num#16,dispatching_base_num#17,pickup_datetime#18,dropoff_datetime#19,PULocationID#20,DOLocationID#21,SR_Flag#22] csv
         +-

In [ ]:
spark.stop()